In [3]:
# Cargo las librerias que voy a utilizar.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
import pickle
import joblib
from sklearn.decomposition import randomized_svd
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# Leo el CSV que voy a utilizar para realizar el modelo junto a las columnas que voy a utilizar.
df = pd.read_csv("plataformas.csv", usecols=['id', 'type', 'title', 'listed_in', 'score'])

In [5]:
df['title'] = df['title'].str.strip()

In [6]:
# Realizo unos vistazos rapidos.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         22998 non-null  object 
 1   type       22998 non-null  object 
 2   title      22998 non-null  object 
 3   listed_in  22998 non-null  object 
 4   score      22998 non-null  float64
dtypes: float64(1), object(4)
memory usage: 898.5+ KB


In [7]:
df.head(2)

,id,type,title,listed_in,score
0,as1,movie,the grand seduction,"comedy, drama",3.5
1,as2,movie,take care good night,"drama, international",3.5


In [8]:
df = df[df['type']== 'movie'].reset_index()
df_corr = df[df['type']== 'movie'].reset_index()

In [9]:
df_corr['listed_in'].unique()

array(['comedy, drama', 'drama, international', 'action, drama, suspense',
       ..., 'action & adventure, documentaries, sports movies',
       'international movies, lgbtq movies, romantic movies',
       'cult movies, dramas, thrillers'], dtype=object)

In [10]:
valores_unicos = df['listed_in'].value_counts()

print(valores_unicos)

drama                             917
documentaries                     508
comedy                            489
comedy, drama                     372
dramas, international movies      362
                                 ... 
comedy, mystery                     1
family, fantasy, romance            1
documentary, family, reality        1
fantasy, musical, thriller          1
cult movies, dramas, thrillers      1
Name: listed_in, Length: 1114, dtype: int64


In [11]:
df.isnull().sum()

index        0
id           0
type         0
title        0
listed_in    0
score        0
dtype: int64

In [12]:
df_corr.isnull().sum()

level_0      0
index        0
id           0
type         0
title        0
listed_in    0
score        0
dtype: int64

In [13]:
categorias = {
    'comedy': ['comedies', 'sports movies','comedy', 'sitcom', 'romantic comedy','stand', 'stand up','up comedy','parody','dramas', 'drama', 'romance','coming of age','teen', 'reality','romance', 'romantic comedy', 'romantic movies'],
    'drama': ['dramas', 'drama','sports movies', 'romance','coming of age','teen', 'reality','romance', 'romantic comedy', 'romantic movies','comedies', 'comedy', 'sitcom', 'romantic comedy','stand', 'stand up','up comedy','parody'],
    'action': ['action', 'action adventure', 'superhero','military and war','spyespionage','western','adventure', 'western','action adventure','superhero', 'survival',
               'sci-fi', 'science fiction','sci'],
    'adventure': ['adventure', 'western','action adventure','superhero', 'survival','western','action', 'action adventure', 'superhero','military and war','spyespionage','mystery','crime','western',
                  'sci-fi', 'science fiction','sci'],
    'thriller': ['thrillers', 'suspense','crime', 'cult movies','horror', 'horror movies', 'thriller','mystery'],
    'sports': ['sports movies', 'sports','dance','fitness', 'health wellness'],
    'documentaries': ['documentaries','sports movies', 'docuseries','animals nature','anthology','arts', 'biographical','arthouse','concert film','disaster','medical', 
                      'documentary','historical', 'history','lifestyle', 'lifestyle culture', 'science technology','special interest'],
    'science fiction': ['sci-fi', 'science fiction','sci','action', 'action adventure', 'superhero','military and war','spyespionage','western','adventure', 'western','action adventure','superhero', 'survival',                        
                        'thrillers', 'suspense','crime', 'cult movies','horror', 'horror movies', 'thriller','mystery','fantasy', 'fi fantasy','anime', 'anime series', 'anime features','animation', 'cartoons','adult animation'],
    'anime':['anime', 'anime series', 'anime features'],
    'animation':['animation', 'cartoons','adult animation'],
    'fantasy':['fantasy', 'fi fantasy','anime', 'anime series', 'anime features','animation', 'cartoons','adult animation'],
    'others': ['movies','and culture','children family movies','classic movies', 'classics', 'cooking food','entertainment','faith and spirituality', 'faith spirituality', 'family','independent movies',
               'international', 'international movies', 'kids', 'latino', 'lgbtq', 'lgbtq movies','music','music musicals','musicals', 'music videos and concerts', 'musical', 
               'talk show and variety','unscripted','variety','young adult audience','news','black stories', 'buddy']
}


In [14]:
def categorias_claves(categorias_dic, categoria_lista):
    claves = []
    for key, values in categorias_dic.items():
        if set(categoria_lista).intersection(values):
            claves.append(key)
    return claves

df_corr['listed_in'] = df_corr['listed_in'].str.lower().str.split(', ')
df_corr['categorias'] = df_corr['listed_in'].apply(lambda x: categorias_claves(categorias, x))
df_corr['categorias'] = df_corr['categorias'].apply(lambda x: ', '.join(x))

In [15]:
df_corr.columns

Index(['level_0', 'index', 'id', 'type', 'title', 'listed_in', 'score',
       'categorias'],
      dtype='object')

In [16]:
df_corr.drop(columns=['level_0', 'index', 'id','type','listed_in'],inplace= True )

In [17]:
"""# crea las columnas dummies
df_corr[['comedy', 'drama', 'action', 'adventure', 'thriller', 'sports', 'documentaries', 'science fiction', 'anime', 'animation', 'fantasy', 'others']] = 0

# itera sobre las columnas dummies y verifica si la columna "categorias" contiene la subcadena correspondiente
for col in df_corr[['comedy', 'drama', 'action', 'adventure', 'thriller', 'sports', 'documentaries', 'science fiction', 'anime', 'animation', 'fantasy', 'others']]:
    df_corr[col] = df_corr['categorias'].str.contains(col, case=False).astype(int)"""


'# crea las columnas dummies\ndf_corr[[\'comedy\', \'drama\', \'action\', \'adventure\', \'thriller\', \'sports\', \'documentaries\', \'science fiction\', \'anime\', \'animation\', \'fantasy\', \'others\']] = 0\n\n# itera sobre las columnas dummies y verifica si la columna "categorias" contiene la subcadena correspondiente\nfor col in df_corr[[\'comedy\', \'drama\', \'action\', \'adventure\', \'thriller\', \'sports\', \'documentaries\', \'science fiction\', \'anime\', \'animation\', \'fantasy\', \'others\']]:\n    df_corr[col] = df_corr[\'categorias\'].str.contains(col, case=False).astype(int)'

In [18]:
#df_corr = df_corr.drop(columns=['listed_in','categorias','type'])

In [19]:
"""df_corr['score'] = df_corr['score'].astype(str).str.replace(".", "")
df_corr['score'] = df_corr['score'].astype(int)
df_corr.columns = df_corr.columns.str.strip()"""

'df_corr[\'score\'] = df_corr[\'score\'].astype(str).str.replace(".", "")\ndf_corr[\'score\'] = df_corr[\'score\'].astype(int)\ndf_corr.columns = df_corr.columns.str.strip()'

In [20]:
df_corr

,title,score,categorias
0,the grand seduction,3.5,"comedy, drama"
1,take care good night,3.5,"comedy, drama, others"
2,secrets of deception,3.5,"comedy, drama, action, adventure, thriller, sc..."
3,pink: staying true,3.5,documentaries
4,monster maker,3.5,"comedy, drama, science fiction, fantasy"
...,...,...,...
16476,zinzana,3.6,"comedy, drama, thriller, science fiction, others"
16477,zodiac,3.4,"comedy, drama, thriller, science fiction"
16478,zombieland,3.4,"comedy, drama, thriller, science fiction"
16479,zoom,3.6,"comedy, drama"


In [21]:
df_corr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16481 entries, 0 to 16480
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       16481 non-null  object 
 1   score       16481 non-null  float64
 2   categorias  16481 non-null  object 
dtypes: float64(1), object(2)
memory usage: 386.4+ KB


In [22]:
df['title'].value_counts()

let there be light    3
zapped                3
mud                   3
good kisser           3
bratz: the movie      3
                     ..
six weeks             1
sivappu malli         1
sideways              1
server sundaram       1
zubaan                1
Name: title, Length: 15943, dtype: int64

In [23]:
df_corr

,title,score,categorias
0,the grand seduction,3.5,"comedy, drama"
1,take care good night,3.5,"comedy, drama, others"
2,secrets of deception,3.5,"comedy, drama, action, adventure, thriller, sc..."
3,pink: staying true,3.5,documentaries
4,monster maker,3.5,"comedy, drama, science fiction, fantasy"
...,...,...,...
16476,zinzana,3.6,"comedy, drama, thriller, science fiction, others"
16477,zodiac,3.4,"comedy, drama, thriller, science fiction"
16478,zombieland,3.4,"comedy, drama, thriller, science fiction"
16479,zoom,3.6,"comedy, drama"


In [24]:
# crear una instancia de TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

# crear la matriz de frecuencia de términos
tfidf_matrix = tfidf.fit_transform(df_corr['categorias'])


In [25]:
# calcular la similitud de coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [26]:
# obtener los índices de las películas más similares
similar_movies = list(enumerate(cosine_sim[10]))

# ordenar las películas por similitud
sorted_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

# obtener los índices de las 5 películas más similares
top_similar = [i[0] for i in sorted_movies[1:6]]

# imprimir las 5 películas más similares
print(df['title'].iloc[top_similar])


17                       zoe
39                yellowneck
56    winds of the wasteland
66                wild geese
74               white tiger
Name: title, dtype: object


In [27]:
def get_recommendations(titulo_pelicula: str):
    
    # Crear el vectorizador TF-IDF
    vectorizer = TfidfVectorizer()

    # Aplicar el vectorizador a la columna 'listed_in' para obtener la matriz de características
    tfidf_matrix = vectorizer.fit_transform(df_corr['categorias'])

    # Calcular la matriz de similitud de coseno
    cosine_sim = cosine_similarity(tfidf_matrix)

    # Obtener el índice de la película que coincide con el título proporcionado
    idx = df.index[df_corr['title'] == titulo_pelicula.lower()].tolist()[0]

    # Obtener las puntuaciones de similitud de coseno de la película con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar las películas según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener las cinco películas más similares, excluyendo la película de entrada
    sim_scores = [i for i in sim_scores if df.index[i[0]] != idx]
    sim_scores = sim_scores[:5]

    # Obtener los títulos de las cinco películas más similares
    recomendaciones = df.iloc[[i[0] for i in sim_scores]]['title'].tolist()

    # Devolver las recomendaciones en el formato especificado
    return {'recomendacion': recomendaciones}


In [28]:
get_recommendations('into the forest')


{'recomendacion': ['take care good night',
  'yatra (telugu)',
  'yatra (malayalam)',
  'where hands touch',
  'wedding cha shinema']}

In [29]:
get_recommendations('the grand seduction')

{'recomendacion': ['grease live!',
  "david's mother",
  'take care',
  "summer '03",
  'zandalee']}

In [30]:
get_recommendations('tarzan')

{'recomendacion': ['stay tuned',
  'technotise: edit & i',
  'on happiness road',
  'the congress',
  'shrek 2']}

In [31]:
df_corr[df_corr['title']=='america the beautiful']

,title,score,categorias
7823,america the beautiful,3.5,


In [32]:
espacios_vacios = (df_corr['categorias'] == ' ').sum()
espacios_vacios

0

In [33]:
# Utilizando el método strip() para eliminar espacios en blanco al inicio y al final de cada valor de la columna
df_corr['categorias'] = df_corr['categorias'].apply(lambda x: x.strip())

# Utilizando el método str.contains() y una expresión regular para buscar valores que contengan espacios en blanco
espacios_vacios =df_corr['categorias'].str.contains('\s', regex=True).sum()

if espacios_vacios > 0:
    print('La columna tiene espacios en blanco.')
else:
    print('La columna no tiene espacios en blanco.')

La columna tiene espacios en blanco.


In [34]:
df_corr.to_csv('ML.csv', index=False, encoding="utf-8")

In [35]:
df_corr.isnull().sum()

title         0
score         0
categorias    0
dtype: int64

In [36]:
df_corr = pd.read_csv('ML.csv')

In [37]:
df_corr.isnull().sum()

title           0
score           0
categorias    802
dtype: int64

In [38]:
# Llenar los valores nulos de la columna 'nombre_columna' con la palabra 'others'
df_corr['categorias'] = df_corr['categorias'].fillna('others')


In [39]:
nulls = df_corr['categorias'].isnull()

filas_con_nulls = df_corr[nulls]

print(filas_con_nulls)

Empty DataFrame
Columns: [title, score, categorias]
Index: []


In [40]:
df_corr.to_csv('ML.csv', index=False, encoding="utf-8")

In [41]:
df_corr = pd.read_csv('ML.csv')

In [42]:
df_corr.isnull().sum()

title         0
score         0
categorias    0
dtype: int64

In [43]:
#df_corr = pd.read_csv('ML.csv')
# Crear el vectorizador TF-IDF
vectorizer = TfidfVectorizer()
# Aplicar el vectorizador a la columna 'listed_in' para obtener la matriz de características
tfidf_matrix = vectorizer.fit_transform(df_corr['categorias'])
# Calcular la matriz de similitud de coseno
cosine_sim = cosine_similarity(tfidf_matrix)

In [44]:
def get_recommendations(title: str):
    
    df_corr = pd.read_csv('ML.csv')
    # Crear el vectorizador TF-IDF
    vectorizer = TfidfVectorizer()
    # Aplicar el vectorizador a la columna 'listed_in' para obtener la matriz de características
    tfidf_matrix = vectorizer.fit_transform(df_corr['categorias'])
    # Calcular la matriz de similitud de coseno
    cosine_sim = cosine_similarity(tfidf_matrix) 
    idx = df_corr.index[df_corr['title'] == title.lower()].tolist()[0]

    # Obtener las puntuaciones de similitud de coseno de la película con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar las películas según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener las cinco películas más similares, excluyendo la película de entrada
    sim_scores = [i for i in sim_scores if df_corr.index[i[0]] != idx]
    sim_scores = sim_scores[:5]

    # Obtener los títulos de las cinco películas más similares
    recomendaciones = df_corr.iloc[[i[0] for i in sim_scores]]['title'].tolist()

    # Devolver las recomendaciones en el formato especificado
    return {'recomendacion': recomendaciones}

In [45]:
get_recommendations('tarzan')

{'recomendacion': ['stay tuned',
  'technotise: edit & i',
  'on happiness road',
  'the congress',
  'shrek 2']}

In [46]:
def matrix():
    
    df_corr = pd.read_csv('ML.csv')
    # Crear el vectorizador TF-IDF
    vectorizer = TfidfVectorizer()
    # Aplicar el vectorizador a la columna 'listed_in' para obtener la matriz de características
    tfidf_matrix = vectorizer.fit_transform(df_corr['categorias'])
    # Calcular la matriz de similitud de coseno
    cosine_sim1 = cosine_similarity(tfidf_matrix)
    return cosine_sim1

In [47]:
azul = 1 + 1

In [48]:

# Guardar el modelo como un archivo pickle
joblib.dump(azul,"modelo.pkl", compress=3)

['modelo.pkl']